# Chatbot with LLM Integration Using Pandas and DialoGPT

This notebook demonstrates the implementation of a customer service chatbot that integrates with datasets and a pre-trained DialoGPT model. The chatbot can respond to customer queries about their information, orders, and product recommendations. Additionally, it tracks response times and collects feedback at the end of the session.

---

# Step 1: Importing Required Libraries

In [1]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import re
import datetime
import string
import time

**Explanation**:  
  - **Pandas** (`pd`) is the main library used to load and manipulate structured data (CSV files) for customers, orders, and products. It will help filter, query, and format data for the chatbot.
  - **Transformers**: We use Hugging Face’s pre-trained **DialoGPT** model, which is loaded using `AutoModelForCausalLM` and `AutoTokenizer`. This allows the chatbot to understand and generate natural language text. The model will be the "brain" for the conversational logic.
  - **Torch**: This deep learning library (PyTorch) is used to handle the tensors passed to the transformer model. It ensures the chatbot can process and generate text using the pre-trained language model.
  - **re**: Regular expressions help to detect user intent and extract relevant entities (like customer IDs, product names, etc.) from the user input.
  - **Datetime**: This is used to manipulate and format dates for customer and order data (e.g., calculating the last login time or order date).
  - **String**: This library is used for text operations, such as cleaning product names by removing punctuation.
  - **Time**: We use this library to track the response time of the chatbot for performance evaluation purposes.

  ---

# Step 2: Loading Datasets

In [2]:
# Load datasets
customers_df = pd.read_csv("../Cleaned_Datasets/customer_SG_only.csv", index_col=0)  # customer data
orders_df = pd.read_csv("../Cleaned_Datasets/orders_generated.csv", index_col=0)  # order data
products_df = pd.read_csv("../Cleaned_Datasets/products_cleaned.csv", index_col=0)  # product data

# Convert customer_id columns to strings in both DataFrames
customers_df['customer_id'] = customers_df['customer_id'].astype(str)
orders_df['customer_id'] = orders_df['customer_id'].astype(str)

**Explanation**:  
  - The datasets for customers, orders, and products are loaded using **Pandas**.  
    - **customers_df**: Contains details such as customer ID, last login, and checkout dates.
    - **orders_df**: Stores order IDs, product IDs, customer IDs, and order times.
    - **products_df**: Contains product titles, prices, and ratings.
  - Customer and order IDs are converted to strings to ensure consistent formatting. This is critical when performing lookups or when comparing IDs across datasets. For example, querying customer data from `customers_df` will require matching the string format of the `customer_id` column.

  ---

# Step 3: Loading Pre-trained DialoGPT Model

In [3]:
# Load pre-trained DialoGPT model and tokenizer (LLM integration)
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

**Explanation**:  
  - The **DialoGPT-medium** variant is loaded, which is a conversational model fine-tuned on dialogue data. It is designed to generate coherent and contextually appropriate responses based on user input.
  - **AutoTokenizer** is responsible for converting the user's input (text) into tokens that the model can process. The model does not understand plain text; it needs the input in a tokenized format.
  - **AutoModelForCausalLM** generates responses based on user input, leveraging the GPT-2 architecture, which is autoregressive, meaning it generates one word at a time based on the previous words in the sentence.
  - By loading the **tokenizer** and **model**, the chatbot is ready to interpret user queries and generate appropriate responses.

---

# Step 4: Context Management Setup

In [4]:
# Context management for follow-up queries
context = {
    "last_product": None,  # Stores the last product mentioned
    "last_intent": None,   # Stores the last intent (price, rating, etc.)
    "last_customer": None, # Stores last customer ID
    "last_order": None     # Stores last order ID
}

**Explanation**:  
  - The chatbot maintains a **context** to handle follow-up questions more naturally. The context stores information about the last product, last intent (such as price or rating), last customer, and last order ID.
  - This allows the chatbot to respond to follow-up queries without needing the user to repeat information. For example, if the user asks about the price of a product and later asks for its rating, the chatbot can refer to the **last_product** in the context and provide the rating seamlessly.

---

# Step 5: LLM (DialoGPT) Chat Handling

In [5]:
# LLM (DialoGPT) Chat Handling
def chat_with_dialoGPT(user_input, chat_history_ids=None):
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
    bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1) if chat_history_ids is not None else new_input_ids
    attention_mask = torch.ones(bot_input_ids.shape, dtype=torch.long)
    
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        attention_mask=attention_mask,
        do_sample=True,
        top_p=0.92,
        top_k=50,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response, chat_history_ids

**Explanation**:  
  - **User Input Tokenization**: The function begins by **tokenizing** the user input (converting text into numerical tokens) using the `tokenizer.encode()` method. It appends an **end-of-sequence (eos) token** to mark the end of the user input.
  - **Handling Conversation History**: If there is any previous chat history (`chat_history_ids`), it concatenates the new input to the existing conversation to maintain context. This enables the chatbot to respond coherently within the flow of an ongoing conversation.
  - **Generating Responses**: The **DialoGPT model** is used to generate a response based on the tokens passed. The **sampling parameters** (`top_p`, `top_k`) control how diverse the generated responses will be. The model continues generating until it reaches the `max_length` or encounters the **pad token**.
  - **Decoding the Response**: The tokens generated by the model are decoded back into a human-readable string using the `tokenizer.decode()` method, which is then returned along with the updated chat history.

  ---

# Step 6: Get Customer Information

In [6]:
# Function to get customer information
def get_customer_info(customer_id):
    customer_id = str(customer_id)
    customers_df['customer_id'] = customers_df['customer_id'].astype(str)
    customer = customers_df[customers_df['customer_id'] == customer_id]
    
    if not customer.empty:
        last_login_days_ago = customer['last_login_day'].values[0]
        last_login_date = (datetime.datetime.now() - datetime.timedelta(days=int(last_login_days_ago))).strftime('%Y-%m-%d')
        checkout_message = "This customer has never checked out."
        
        last_checkout_days_ago = customer['last_checkout_day'].values[0]
        if last_checkout_days_ago.isdigit():
            last_checkout_date = (datetime.datetime.now() - datetime.timedelta(days=int(last_checkout_days_ago))).strftime('%Y-%m-%d')
            checkout_message = f"and their most recent checkout was {last_checkout_days_ago} days ago on {last_checkout_date}."
        
        context["last_customer"] = customer_id  # Store for future queries
        return f"Customer {customer_id} last logged in {last_login_days_ago} days ago on {last_login_date} {checkout_message}"
    
    return "Could not find the customer. Please check the ID!"

**Explanation**:  
  - The `customer_id` is converted to a string and matched against the **customers_df** dataframe to retrieve customer details.
  - **Last Login Calculation**: If the customer is found, the function calculates how many days ago the customer logged in based on the `last_login_day` column and formats it as a readable date.
  - **Checkout Status**: The function checks if the customer has ever checked out. If they have, it calculates the last checkout date; otherwise, it returns a message stating that the customer has never checked out.
  - **Context Update**: The `customer_id` is stored in the **context** for future use, ensuring that follow-up queries can reference the same customer without needing the ID to be repeated.

---

# Step 7: Product Recommendation Based on Customer History

In [7]:
# Function to recommend products based on customer history 
def recommend_products(customer_id):
    customer_id = str(customer_id)  # Ensure customer_id is a string
    
    # Get customer's past orders
    customer_orders = orders_df[orders_df['customer_id'] == customer_id]
    
    if customer_orders.empty:
        # If no orders, select 3 popular products
        product_recommendations = products_df[['title', 'price_actual']].sample(3)
        return "This customer has no past orders. Here are some popular recommendations:\n" + \
               product_recommendations.to_string(index=False)
    
    # Get the list of product IDs that the customer has already purchased
    purchased_product_ids = customer_orders['product_id'].unique()
    
    # Recommend products that the customer hasn't purchased yet
    available_products = products_df[~products_df['product_id'].isin(purchased_product_ids)]
    
    if available_products.empty:
        return "No new product recommendations at the moment."
    
    # Select 3 products from the remaining ones
    recommended_products = available_products.sample(min(3, len(available_products)))
    
    # Format the output
    recommendations = recommended_products[['title', 'price_actual']].to_string(index=False)
    return f"Based on your purchase history, I recommend these products:\n{recommendations}"

**Explanation**:  
  - **Customer Orders Lookup**: The function retrieves all orders placed by the customer from the `orders_df` dataframe. If the customer has no orders, the chatbot selects 3 random popular products and presents them as recommendations.
  - **Product Recommendations**: If the customer has previous orders, the function identifies which products have already been purchased and recommends products the customer hasn't bought yet.
  - **Sampling Products**: From the remaining available products, the chatbot selects up to 3 random products to recommend. These are formatted and presented to the user.
  - This logic ensures that the chatbot provides relevant product recommendations based on the customer’s purchase history.

  ---

# Step 8: Order Status Function

In [8]:
# Function to get order status
def get_order_status(order_id):
    order_id = str(order_id)  # Ensure order ID is a string
    orders_df['order_id'] = orders_df['order_id'].astype(str)

    order = orders_df[orders_df['order_id'] == order_id]
    if not order.empty:
        product_id = order['product_id'].values[0]
        order_time = order['order_time'].values[0]
        context["last_order"] = order_id  # Store order ID for follow-up
        return (f"Good news! Order {order_id} was placed on {order_time}. "
                f"The order includes product {product_id}. ")
    else:
        return (f"Oops, it seems there’s no order with ID {order_id}. "
                "Can you please check the ID and try again? I’m here to help!")

**Explanation**:  
  - **Order Lookup**: The function retrieves the order details by matching the `order_id` in the `orders_df` dataframe.
  - **Order Status**: If the order is found, the chatbot returns details such as the order time and the associated product ID. If the order isn’t found, it prompts the user to check the order ID.
  - **Context Update**: The `order_id` is stored in the **context** for follow-up queries.

---

# Step 9: Product Information (Price/Rating) Handling

In [9]:
# Function to handle product-related queries (price or rating)
def get_product_info(product_name, info_type="price"):
    # Clean up the product name by removing punctuation
    product_name_cleaned = product_name.translate(str.maketrans('', '', string.punctuation))
    
    # Split the user input into individual keywords
    keywords = product_name_cleaned.split()
    
    # Perform a case-insensitive search for rows that contain all keywords in the title
    matching_products = products_df[
        products_df['title'].apply(lambda x: all(kw.lower() in x.lower() for kw in keywords))
    ]
    
    if not matching_products.empty:
        # Get the first match (can be expanded to multiple matches if needed)
        product = matching_products.iloc[0]
        price = product['price_actual']
        rating = product['item_rating']
        title = product['title']
        
        # Store product context for follow-up
        context["last_product"] = title  # Use the product title for follow-up queries
        context["last_intent"] = info_type  # Whether the user asked for price or rating
        
        # Provide appropriate information based on info_type
        if info_type == "price":
            return (f"I found the product you're looking for! The '{title}' is priced at {price}. ")
        elif info_type == "rating":
            return (f"The '{title}' has a rating of {rating} stars.")
    else:
        return f"I couldn't find any product containing the keywords '{product_name}'. Please try different keywords!"

**Explanation**:  
  - **Product Lookup**: The product name provided by the user is cleaned (removing punctuation) and split into keywords. A case-insensitive search is performed to match these keywords against product titles in the `products_df` dataframe.
  - **Return Price or Rating**: Based on the user's query (`info_type`), the chatbot returns either the product's price or rating. If the product isn't found, the chatbot asks the user to try different keywords.
  - **Context Update**: The product name and intent are stored in the **context** for future reference (e.g., follow-up queries).

  ---

# Step 10: Intent Detection and Entity Extraction

In [10]:
# Regex-based intent detection
def detect_intent_and_entities(user_input):
    # Prioritize recommendation detection over customer queries
    if "recommend" in user_input.lower() and "customer" in user_input.lower():
        customer_id = re.search(r'\d+', user_input).group(0)
        return "recommendation", {"customer_id": customer_id}

    # Regex patterns to detect customer query intent
    if re.search(r'customer.*id.*\d+', user_input, re.IGNORECASE):
        intent = "customer_query"
        customer_id = re.search(r'\d+', user_input).group(0)
        return intent, {"customer_id": customer_id}
    
    # Regex patterns to detect order query intent
    if re.search(r'order.*id', user_input, re.IGNORECASE):
        intent = "order_query"
        order_id = re.search(r'order_[0-9]+', user_input, re.IGNORECASE)
        return intent, {"order_id": order_id.group(0)} if order_id else {"order_id": None}
    
    # Regex patterns to detect product price or rating queries
    if re.search(r'price|rating', user_input, re.IGNORECASE):
        product_name_match = re.search(r'(?:price|rating) of (.+)', user_input, re.IGNORECASE)
        product_name = product_name_match.group(1).strip() if product_name_match else None
        intent = "product_price" if "price" in user_input.lower() else "product_rating"
        return intent, {"product_name": product_name}

    # Fallback for general queries
    return "general", {}

**Explanation**:  
  - **Intent Detection**: The function uses regular expressions to detect the user’s intent (e.g., whether they’re asking for customer information, an order status, or product details).
  - **Entity Extraction**: The function extracts relevant entities like **customer IDs**, **order IDs**, or **product names** based on the detected intent. For example, if a user asks for the price of a product, the product name is extracted from the input.
  - This process ensures the chatbot understands the user’s query and can handle it appropriately.

  ---

# Step 11: Collecting Customer Feedback

In [11]:
# Function to collect customer feedback after the session
def request_feedback():
    feedback = input("Bot: How would you rate your experience on a scale from 1 to 5? ")
    if feedback.isdigit() and 1 <= int(feedback) <= 5:
        return int(feedback)
    return None

**Explanation**:  
  - This function prompts the user to provide feedback after the session. The user is asked to rate their experience on a scale from 1 to 5.
  - **Input Validation**: The function checks if the input is a digit between 1 and 5 using the `isdigit()` method and returns the feedback score if it's valid.
  - If the input is invalid (not a number between 1 and 5), the function returns `None`, meaning no valid feedback was collected.
  - The collected feedback can be stored and analyzed later to assess user satisfaction with the chatbot’s performance.

  ---

# Step 12: Chatbot Main Function with LLM and Metrics Collection (The chatbot's output will be displayed after this section)

In [ ]:
# Chatbot core function with LLM and metrics collection
def chatbot():
    print("Hi there! I'm your friendly chatbot, here to help you with any queries. Type 'quit' to exit the chat.")
    chat_history_ids = None  # Initialize chat_history_ids to None at the start
    total_queries = 0  # To track the number of queries handled
    response_times = []  # To track response times for efficiency evaluation
    feedback_scores = []  # To track user feedback ratings
    
    while True:
        user_input = input("You: ").strip().lower()
        
        # Check if the user wants to quit
        if user_input == 'quit':
            # Ask for feedback before quitting
            feedback = request_feedback()
            if feedback:
                feedback_scores.append(feedback)
                print(f"Thank you for your feedback! You rated this session {feedback}/5.")
            
            # Say goodbye and break the loop
            print("Goodbye! Feel free to ask for assistance anytime!")
            break

        # Start the timer to measure response time
        start_time = time.time()

        # Enhanced Natural Language Processing for intent and entity detection
        intent, entities = detect_intent_and_entities(user_input)

        # Handle customer queries
        if intent == "customer_query":
            customer_id = entities.get("customer_id")
            if customer_id:
                response = get_customer_info(customer_id)
            else:
                response = "It seems like you asked about a customer, but I couldn't find the ID. Please provide the customer ID."

        # Handle order queries
        elif intent == "order_query":
            order_id = entities.get("order_id")
            if order_id:
                response = get_order_status(order_id)
            else:
                response = "It seems like you asked about an order, but I couldn't find the order ID. Please provide the order ID."

        # Handle product queries based on intent
        elif intent in ["product_price", "product_rating"]:
            product_name = entities.get("product_name")
            
            if product_name:
                # Update product name in the context when a new product is mentioned
                context["last_product"] = product_name.translate(str.maketrans('', '', string.punctuation))
                context["last_intent"] = "price" if intent == "product_price" else "rating"
                
                # Respond with the appropriate information (price or rating)
                response = get_product_info(product_name, info_type=context["last_intent"])
            else:
                # If product name is not explicitly mentioned, check if there's a follow-up on the last product
                if context.get("last_product"):
                    product_name = context["last_product"]
                    # Update intent based on user input (switch between price or rating)
                    if "price" in user_input:
                        context["last_intent"] = "price"
                    elif "rating" in user_input:
                        context["last_intent"] = "rating"
                    
                    # Provide the appropriate information based on the updated intent
                    response = get_product_info(product_name, info_type=context["last_intent"])
                else:
                    response = "Could you please specify which product you're asking about?"

        # Handle product recommendations
        elif intent == "recommendation":
            customer_id = entities.get("customer_id")
            if customer_id:
                # First, get customer info
                customer_info = get_customer_info(customer_id)
                
                # Then, provide recommendations
                recommendations = recommend_products(customer_id)
                
                # Combine the two responses
                response = f"{customer_info}\n{recommendations}"
            else:
                response = "Please provide a customer ID to get recommendations."

        # Handle follow-up queries on the last product
        elif "price" in user_input and context.get("last_product"):
            # User asks for price after a previous query (like rating)
            product_name = context["last_product"]
            response = get_product_info(product_name, info_type="price")
            context["last_intent"] = "price"  # Update the intent to price

        elif "rating" in user_input and context.get("last_product"):
            # User asks for rating after a previous query (like price)
            product_name = context["last_product"]
            response = get_product_info(product_name, info_type="rating")
            context["last_intent"] = "rating"  # Update the intent to rating
        
        else:
            # Fallback to DialoGPT if no specific intent was detected
            response, chat_history_ids = chat_with_dialoGPT(user_input, chat_history_ids)

        # Stop the timer and calculate the response time
        end_time = time.time()
        response_time = end_time - start_time
        response_times.append(response_time)

        # Track the number of queries
        total_queries += 1

        # Print the chatbot's response
        print(f"Bot: {response}")

    # Calculate average response time and feedback scores for efficiency and satisfaction metrics
    if response_times:
        avg_response_time = sum(response_times) / len(response_times)
        print(f"\nAverage response time: {avg_response_time:.2f} seconds")

    if feedback_scores:
        avg_feedback_score = sum(feedback_scores) / len(feedback_scores)
        print(f"Average feedback rating: {avg_feedback_score:.1f}/5")

# Start the chatbot
chatbot()

**Explanation**:  
  - **Core Chatbot Logic**: This function handles the chatbot’s main conversation loop. It continuously prompts the user for input and determines the intent of the query using the `detect_intent_and_entities` function.
  - **Tracking Response Times**: The time taken to generate a response is tracked and stored in the `response_times` list. This helps assess the chatbot’s performance.
  - **User Feedback Collection**: Before the user exits the conversation, they are asked to rate their experience (on a scale of 1 to 5). The feedback is stored and can be used to evaluate user satisfaction.
  - **Handling Various Intents**: The chatbot handles customer queries, order statuses, product prices/ratings, and product recommendations based on the detected intent. If no specific intent is detected, it uses **DialoGPT** to generate a general response.
  - The final step calls the `chatbot()` function to start the conversation. This activates the chatbot, allowing it to process user queries, respond based on intents, and collect performance metrics such as response times and user feedback.

  ---

## Chatbot Testing Scenarios

### **Customer-Related Queries:**

**Question 1**

- **User**: What’s the last login for **customer ID** 104511?

- **Bot**: Great news! We found the information for **customer ID** 104511. They last logged in 4 days ago on 2024-10-23, and their most recent checkout was 4 days ago on 2024-10-23.


**Question 2**

- **User**: Can you check the details for **customer ID** 20596?

- **Bot**: Great news! We found the information for **customer ID** 20596. They last logged in 64 days ago on 2024-08-24, and this customer has never checked out.


**Question 3**

- **User**: Tell me the last checkout date for **customer ID** 119330!

- **Bot**: Great news! We found the information for **customer ID** 119330. They last logged in 12 days ago on 2024-10-15, and their most recent checkout was 18 days ago on 2024-10-09.


**Question 4**

- **User**: When was the last login for **customer ID** 46400?

- **Bot**: Great news! We found the information for **customer ID** 46400. They last logged in 295 days ago on 2024-01-06, and this customer has never checked out.

---

### **Order-Related Queries:**

**Question 1**

- **User**: What is the status of **order ID** order_123?

- **Bot**: Good news! **Order ID** order_123 was placed on 2019-06-25 16:16:43.205354. The order includes product 11756.


**Question 2**

- **User**: Can you check the order status for **order ID** order_987?

- **Bot**: Good news! **Order ID** order_987 was placed on 2019-10-12 22:04:03.829836. The order includes product 4533.


**Question 3**

- **User**: Tell me the order time for **order ID** order_928!

- **Bot**: Good news! **Order ID** order_928 was placed on 2019-09-10 20:11:41.736175. The order includes product 6108.


**Question 4**

- **User**: What product is included in **order ID** order_65893?

- **Bot**: Good news! **Order ID** order_65893 was placed on 2019-03-25 15:29:08.967064. The order includes product 4471.

---

### **Product-Related Queries (Price):**

**Question 1**

- **User**: What’s **the price of** shoulder bag?

- **Bot**: I found the product you're looking for! The '1081 PU SLING BAG KULIT leather Casual Beg Tangan Wanita women Shoulder Wallet Purse Travel READY STOCK QQF WHOLESALE | Shopee Malaysia' is priced at 4.017214855598254.


**Question 2**

- **User**: Can you find **the price of** wireless keyboard?

- **Bot**: I found the product you're looking for! The 'NIXXOS For iPad10 Wireless Keyboard Ccase for 10.2'' iPad 7th/8th/9th Gen 10.9'' Air4/Air5 2021/2020 Pro 11'' 7-color Backlight Magnetic Keyboard Leather Cover | Shopee Malaysia' is priced at 21.487428297386007.


**Question 3**

- **User**: Tell me **the price of** shirt.

- **Bot**: I found the product you're looking for! The 'Men Male Shirt Solid Blouse Jumper Shirts Pullover T-Shirt High neck Basic Thermal Mens Casual Long Sleeve Tops | Shopee Malaysia' is priced at 1.8684720258596528.


**Question 4**

- **User**: How much is **the price of** shoes?

- **Bot**: I found the product you're looking for! The 'Fashion Women Flat Shoes Suede Leather Colth Soft Ladies Flat Heel Working Shoe Pure Colours | Shopee Malaysia' is priced at 5.6209866777944555.


**Question 5**

- **User**: Find **the price of** apple pencil!

- **Bot**: I found the product you're looking for! The 'Apple Pencil (2nd Generation) | Shopee Malaysia' is priced at 139.823989935164.

---

### **Product-Related Queries (Rating):**

**Question 1**

- **User**: What is **the rating of** shoulder bag?

- **Bot**: The '1081 PU SLING BAG KULIT leather Casual Beg Tangan Wanita women Shoulder Wallet Purse Travel READY STOCK QQF WHOLESALE | Shopee Malaysia' has a rating of 4.9 stars.


**Question 2**

- **User**: Can you find **the rating of** wireless keyboard?

- **Bot**: The 'NIXXOS For iPad10 Wireless Keyboard Ccase for 10.2'' iPad 7th/8th/9th Gen 10.9'' Air4/Air5 2021/2020 Pro 11'' 7-color Backlight Magnetic Keyboard Leather Cover | Shopee Malaysia' has a rating of 4.9 stars.


**Question 3**

- **User**: What’s **the rating of** shirt?

- **Bot**: The 'Men Male Shirt Solid Blouse Jumper Shirts Pullover T-Shirt High neck Basic Thermal Mens Casual Long Sleeve Tops | Shopee Malaysia' has a rating of 4.8 stars.


**Question 4**

- **User**: Tell me **the rating of** shoes.

- **Bot**: The 'Fashion Women Flat Shoes Suede Leather Colth Soft Ladies Flat Heel Working Shoe Pure Colours | Shopee Malaysia' has a rating of 4.9 stars.


**Question 5**

- **User**: Find **the rating of** apple pencil!

- **Bot**: The 'Apple Pencil (2nd Generation) | Shopee Malaysia' has a rating of 4.9 stars.

---

### **Follow-Up Product Queries:**

**Conversation 1**

- **User**: What’s **the price of** apple pencil?

- **Bot**: I found the product you're looking for! The 'Apple Pencil (2nd Generation) | Shopee Malaysia' is priced at 139.823989935164.

- **User**: What is its rating? *(User can also ask: What is the rating?)*

- **Bot**: The 'Apple Pencil (2nd Generation) | Shopee Malaysia' has a rating of 4.9 stars.


**Conversation 2**

- **User**: Tell me **the rating of** the shoulder bag.

- **Bot**: The '1081 PU SLING BAG KULIT leather Casual Beg Tangan Wanita women Shoulder Wallet Purse Travel READY STOCK QQF WHOLESALE | Shopee Malaysia' has a rating of 4.9 stars.

- **User**: How much is its price? *(User can also ask: What about its price?)*

- **Bot**: I found the product you're looking for! The '1081 PU SLING BAG KULIT leather Casual Beg Tangan Wanita women Shoulder Wallet Purse Travel READY STOCK QQF WHOLESALE | Shopee Malaysia' is priced at 4.017214855598254.


**Conversation 3**

- **User**: What is **the price of** wireless keyboard?

- **Bot**: I found the product you're looking for! The '10 inch Mini Bluetooth Wireless Keyboard And Mouse Set Air Slim Keyboard For Phone Android Windows Tablet 蓝牙无线键盘 | Shopee Malaysia' is priced at 3.422417927366264.

- **User**: What about its rating? *(User can also ask: Tell me about its rating)*

- **Bot**: The '10 inch Mini Bluetooth Wireless Keyboard And Mouse Set Air Slim Keyboard For Phone Android Windows Tablet 蓝牙无线键盘 | Shopee Malaysia' has a rating of 4.9 stars.

---

### **Product Recommendation Queries:**

Please note that the answers in this section are randomized but are still based on the customer's purchase history. Products that have already been purchased will not be included.

**Note**: In the actual chatbot output, the product titles and prices are displayed in a continuous text format, side by side, rather than as separate lines or in a table. The layout follows a natural conversational flow, where product names and their prices are presented next to each other in a more compact, real-time manner. This format reflects how responses will appear in a live chatbot interaction.


**Question 1**

- **User**: Can you give product recommendations for **customer ID** 299?

- **Bot**: Customer 299 last logged in 1 day ago on 2024-10-26 and their most recent checkout was 2 days ago on 2024-10-25.  
  Based on your purchase history, I recommend these products:
  
  title | price_actual
  
  Keychain Key Ring Cartoon Keychain Keyfob Portable Party Key chain Free Gift BABY CITY (Shopee Malaysia)  | 0.031141  
  HIGH CULTURED® Classic Oxford Regular Button-Down Long Sleeve Shirt - 267 (Shopee Malaysia) | 27.995939  
  10pcs (WHITE/GOLD) KIYOME KINOKI Cleansing Detox Foot Pads Pad Kaki Pembersih Detox (Shopee Malaysia)  | 0.155706

**Question 2**

- **User**: Please provide product recommendations for **customer ID** 126576!

- **Bot**: Customer 126576 last logged in 20 days ago on 2024-10-07 and their most recent checkout was 180 days ago on 2024-04-30.  
  Based on your purchase history, I recommend these products:

  title | price_actual

  Ribena Cheerpack - Regular (330ml x 6) (Shopee Malaysia) | 4.655609  
  👍🏻Heavy Duty👍🏻 Car Tyre Tubeless Valve Aluminium /Kepala Tayar Tiub Angin Wheel Nut Sport Rim Nuts Tire Tube Valve TR48E (Shopee Malaysia) | 0.267814  
  Pashaphone Corded Phone KX-T504 with Big Button Landline Telephone TM Unifi Maxis Office Home Telefon 家用电话 (Shopee Malaysia) | 6.197099

**Question 3**

- **User**: Tell me the product recommendations for **customer ID** 105738.

- **Bot**: Customer 105738 last logged in 30 days ago on 2024-09-27 and their most recent checkout was 91 days ago on 2024-07-28.  
  Based on your purchase history, I recommend these products:

  title | price_actual
  
  Apple iPad 10.9-inch 10th Generation 2022 (Wi-Fi) (Shopee Malaysia) | 653.653797  
  (Random Color) 1 PCS Classic Pull Flywheel Flash Yo-Yo Toy Multicolor Light Color Flying Saucer Model Luminous (Shopee Malaysia) | 0.183733  
  Tshirt Dewasa Stand for Palestine / SAVE GAZA (S-XXL) Tshirt Lelaki Perempuan Cotton Lengan Pendek Murah (Shopee Malaysia) | 1.494778

---

### **General Queries (Fallback to DialoGPT):**

- **User**: "Hello!"
  
- **User**: "How are you today?"
  
- **User**: "What’s the weather like?"

---

# Leveraging Large Language Models (LLMs) for Personalized Customer Support: Implementation and Evaluation of an AI Chatbot

Large Language Models (LLMs) have significant potential for enhancing personalized customer support and engagement, particularly when implemented as AI chatbots. Evaluating the effectiveness of these chatbots involves assessing their impact on customer satisfaction and support efficiency.

The following breakdown explains how the chatbot implementation addresses these goals:

### 1. **Implementing an AI Chatbot Using an LLM**

The implementation uses **DialoGPT**, a transformer-based LLM, which is fine-tuned for dialogue generation, making it suitable for customer query handling and product recommendations. Here's how the chatbot functions:

- **LLM Integration:**
  - The `chat_with_dialoGPT` function utilizes the **DialoGPT** model to process user inputs and generate human-like conversational responses. It manages conversation history to ensure context is retained across user interactions, improving engagement by delivering personalized responses.
  
- **Context Management:**
  - The chatbot maintains a `context` dictionary, which stores important information such as the last product or last customer ID discussed. This ensures continuity in conversation, allowing for more personalized responses in follow-up queries.

- **Handling Customer Queries:**
  - **Customer Information Query:** The function `get_customer_info(customer_id)` retrieves customer data, such as last login details and last checkout history. The LLM uses this data to provide relevant responses, enhancing personalization.
  
  - **Order Status Query:** The chatbot can handle queries related to customer orders through `get_order_status(order_id)`, which provides information on the order’s status based on the order ID.
  
- **Product Recommendations:**
  - The function `recommend_products(customer_id)` suggests new products based on a customer’s purchase history. If the customer has no previous orders, it recommends popular products instead. This level of personalization enhances customer engagement and increases the chances of successful product suggestions.

- **Product Information Query (Price/Rating):**
  - Through `get_product_info(product_name, info_type)`, the chatbot can answer questions about product details such as price or rating. It can also follow up on a product from earlier in the conversation, demonstrating the chatbot’s ability to handle specific customer needs effectively.


### 2. **Evaluating the Effectiveness of the AI Chatbot**

The chatbot code includes several metrics and mechanisms to assess its performance:

- **Efficiency (Response Time):**
  - The chatbot calculates **response time** for each query using the `time.time()` function. These response times are stored in the `response_times` list and averaged at the end of the session to evaluate how quickly the chatbot responds to user queries.
  
  - **Tracking Total Queries:** The total number of queries is tracked via the `total_queries` variable, allowing the system to assess the chatbot's workload and performance under various conditions.

- **Customer Satisfaction (Feedback Collection):**
  - At the end of the chat session, the function `request_feedback()` prompts users to rate their experience on a scale from 1 to 5. These feedback scores are stored and averaged, giving a direct measure of **customer satisfaction**.
  
  - An average feedback score is calculated and displayed at the end of the session, providing insight into how users perceive the chatbot’s effectiveness.


### 3. **Potential of LLMs for Personalized Customer Support**

The implementation demonstrates the following potential of using LLMs like **DialoGPT** for personalized customer support:

- **Natural, Human-Like Interactions:**
  - The LLM-based chatbot can engage customers in natural, flowing conversations, responding to a wide range of queries and maintaining context across multiple interactions.

- **Context Awareness:**
  - The ability to retain and reference information from previous queries (e.g., customer history, product details) enhances personalization, allowing the chatbot to provide more relevant and tailored responses.

- **Scalability:**
  - The LLM is capable of handling multiple customer queries at once, making it a scalable solution for customer support operations without the need for additional human agents.

- **Product and Service Recommendations:**
  - By analyzing customer history, the chatbot can offer personalized recommendations, which can improve customer engagement and drive sales.


### 4. **Summary of Chatbot Capabilities and Effectiveness**

- **Personalized Customer Support and Engagement:**
  - The chatbot can tailor responses based on customer history, offer relevant product recommendations, and assist with product inquiries.
  
- **AI Chatbot for Customer Queries:**
  - The chatbot uses LLM capabilities to handle customer queries regarding products, orders, and general inquiries, maintaining a natural conversational flow and engaging users effectively.

- **Evaluation of Effectiveness:**
  - The chatbot tracks metrics such as average response time and feedback scores, allowing for the measurement of support efficiency and customer satisfaction.

This detailed chatbot implementation demonstrates how LLMs can be used for personalized customer support, with mechanisms for evaluating their effectiveness in real-world scenarios.

---

# Potential Benefits of this Chatbot for the E-commerce Platform and Customers

Implementing a chatbot on an e-commerce platform offers significant benefits for both the platform itself and its customers:

#### **1. For the E-commerce Platform:**
- **Increased Efficiency in Customer Support**: By automating responses to common customer queries such as order status, product details (price, ratings), and customer account information, the platform reduces the load on human customer support representatives. This allows the platform to handle more customers simultaneously without increasing operational costs.
  
- **24/7 Availability**: The chatbot provides uninterrupted support, ensuring that customers can get their queries answered at any time, even outside normal business hours. This improves customer satisfaction and potentially increases sales by assisting customers around the clock.

- **Cost Reduction**: Automating a portion of the customer service reduces the need for hiring large teams of support agents. This results in lower staffing and training costs, enabling the platform to allocate resources to other critical areas like marketing or product development.

- **Better Data Collection**: The chatbot logs customer interactions, providing the platform with valuable insights into common customer pain points, frequently asked questions, and product-related concerns. This data can be used to improve the platform's offerings and customer experience.

- **Faster Response Times**: Chatbots can instantly reply to customer inquiries, leading to a smoother and more responsive customer experience. This helps in reducing bounce rates and cart abandonment, as customers are more likely to make purchases when they get prompt assistance.

#### **2. For Customers:**
- **Immediate Assistance**: Customers no longer have to wait in long queues for help. Whether they need to track an order, ask about a product, or get help with their account, the chatbot can provide immediate responses.
  
- **Personalized Experience**: By tracking previous interactions through the chatbot’s **context management**, the platform can provide more personalized responses, such as remembering the last product a customer inquired about or their last order. This personal touch enhances the overall shopping experience.

- **Seamless Shopping Experience**: With quick access to product details, prices, and ratings, customers can make informed purchasing decisions without needing to leave the platform or contact support. This reduces friction in the buying process and can lead to increased sales.

- **Reduced Effort for Repeated Queries**: Customers often need the same information multiple times, such as order tracking. With the chatbot storing context (e.g., last customer, last product), they don’t need to re-enter this information, making the experience more efficient and user-friendly.

---

# Challenges Encountered During Development and How They Were Addressed

Developing this chatbot involved several challenges, ranging from handling natural language processing (NLP) complexities to managing real-time data from multiple datasets. Here’s a breakdown of the key challenges and the solutions implemented to address them:

#### **1. Managing Natural Language Understanding and Intent Detection**
- **Challenge**: One of the primary challenges in developing a chatbot is accurately understanding the user's intent. Users may ask about the same topic (e.g., product price or order status) in various ways, making it difficult for the chatbot to detect the correct intent and respond appropriately.

- **Solution**: We used regular expressions (`regex`) to match specific patterns in the input that signal intent. For instance, the chatbot looks for keywords such as “price of” or “order ID” to determine whether the user is asking about a product or order. This approach, combined with the use of a context dictionary, allows the chatbot to handle follow-up queries and manage different intents effectively.

#### **2. Ensuring Context Awareness for Follow-up Queries**
- **Challenge**: A key aspect of user experience in chatbots is the ability to handle follow-up questions without requiring users to repeat information. For example, if a user asks for the rating of a product and then asks for its price, the chatbot should remember which product was previously mentioned.

- **Solution**: We addressed this challenge by implementing a **context management system**. The chatbot stores the last product, customer, or order mentioned in a conversation in a dictionary (`context`). This way, when a user asks a follow-up question, the chatbot can retrieve the relevant information without needing the user to repeat themselves.

#### **3. Handling Large and Structured Datasets**
- **Challenge**: Since the chatbot interacts with multiple datasets (customers, orders, products), managing the data efficiently and querying it in real-time posed a challenge. The chatbot had to search for specific details like customer IDs, product names, and order IDs without performance issues.

- **Solution**: To ensure efficient data handling, we used **Pandas** for loading and querying the CSV files. This allowed us to manipulate and filter the data quickly. Data columns were pre-processed to ensure uniform formats, and indices were set for faster lookups (e.g., using `customer_id` or `order_id` as indices). This optimization improved the chatbot’s performance when retrieving customer or product information.

#### **4. Generating Conversational Responses with DialoGPT**
- **Challenge**: Creating human-like responses using a pre-trained model (DialoGPT) was another challenge. While DialoGPT generates coherent conversations, it requires proper management of chat history and tokens to maintain the context of the conversation across multiple exchanges.

- **Solution**: We handled this by maintaining a **chat history** (`chat_history_ids`) to track all user inputs and chatbot responses. The model was initialized with each user input, along with the previous conversation history, allowing it to generate responses that fit the ongoing context. Additionally, parameters like **`top_p`** and **`top_k`** were used to control the randomness of the chatbot’s replies, ensuring the responses were both relevant and varied.

#### **5. Handling User Input Variability**
- **Challenge**: Users often phrase their queries differently. For example, someone might ask "How much is this?" or "What’s the price?" when referring to a product. This variability in language makes it difficult to capture all possible user inputs using fixed rules.

- **Solution**: We addressed this challenge by allowing the chatbot to search for keywords (e.g., “price”, “rating”) and applying string manipulation techniques such as removing punctuation and making the search case-insensitive. This increased the flexibility of the chatbot in recognizing user queries. Additionally, when specific information (like a product name) was missing, the chatbot used the **stored context** to infer what the user was referring to.

#### **6. Ensuring Scalability and Adaptability**
- **Challenge**: As the platform grows, more products, customers, and orders will be added to the datasets, increasing the complexity and size of the data that the chatbot needs to handle.

- **Solution**: The chatbot was designed to be easily scalable by allowing dynamic updates to the datasets. Since the bot relies on **Pandas** DataFrames, new rows can be added to the CSV files without breaking functionality. Additionally, pre-trained models like DialoGPT are versatile and can be fine-tuned further for specific domain tasks, allowing the chatbot to adapt as the platform’s offerings evolve.

---